In [1]:
import os, argparse, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    cohen_kappa_score
)

from pygam import LogisticGAM, s, f

In [3]:
def infer_columns(df: pd.DataFrame, target: str = None):
    X = df.copy()

    if target is not None:
        X = X.drop(columns=[target])
    cat_cols = list(X.select_dtypes(include=["object", "category"]).columns)

    # treat low-cardinality integers as categorical
    for c in X.select_dtypes(include=["int64", "int32", "int16", "int8"]).columns:
        if X[c].nunique() <= 20:
            cat_cols.append(c)

    cat_cols = sorted(set(cat_cols))
    num_cols = [c for c in X.columns if c not in cat_cols]
    return num_cols, cat_cols


In [4]:
def plot_confusion_matrix(cm, labels, outpath="gam_confusion_matrix.png", title="Confusion Matrix – GAM"):
    fig, ax = plt.subplots()
    im = ax.imshow(cm)

    ax.set_xticks(range(len(labels))); ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha="right"); ax.set_yticklabels(labels)

    for i in range(len(cm)):
        for j in range(len(cm)):
            ax.text(j, i, cm[i, j], ha="center", va="center")

    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    ax.set_title(title)
    plt.tight_layout()
    fig.savefig(outpath)
    plt.close(fig)

In [5]:
def main():
    # ---- Configuration ----
    class Args:
        csv = "student_data.csv"  # Update if path differs
        target = None             # Auto-detect
        sep = ";"
        test_size = 0.2
        seed = 42
        save_model = "gam_model.joblib"
        pred_out = "gam_predictions.csv"
        cm_out = "gam_confusion_matrix.png"

    args = Args()
    base_path = Path().resolve()
    csv_path = base_path / args.csv
    args.save_model = base_path / args.save_model
    args.pred_out = base_path / args.pred_out
    args.cm_out = base_path / args.cm_out

    # ---- Load CSV ----
    print("Loading CSV from:", csv_path)
    df = pd.read_csv(csv_path, sep=args.sep, engine="python", encoding="utf-8-sig")
    print(f"Loaded {len(df)} rows.")

    # Normalize column names
    df.columns = [str(c).replace("\ufeff", "").strip().replace(" ", "_") for c in df.columns]

    # ---- Detect target column ----
    if args.target is None or args.target.replace(" ", "_") not in df.columns:
        possible_targets = ["Output", "Target", "Status", "Outcome", "Result", "Label"]
        found = [c for c in df.columns if c in possible_targets or c.lower() in [t.lower() for t in possible_targets]]
        if not found:
            raise ValueError("Could not identify target column. Set args.target manually.")
        target_col = found[0]
    else:
        target_col = args.target.replace(" ", "_")
    print("Detected target column:", target_col)

    # ---- Filter binary target ----
    df_binary = df[df[target_col].isin(["Dropout", "Graduate"])].copy()
    print(f"Filtered to binary target: {len(df_binary)} rows")
    y = df_binary[target_col]
    X = df_binary.drop(columns=[target_col])

    # ---- Encode target ----
    from sklearn.preprocessing import LabelEncoder
    label_enc = LabelEncoder()
    y_enc = label_enc.fit_transform(y)

    # ---- Infer numerical and categorical columns ----
    def infer_columns(df: pd.DataFrame):
        cat_cols = list(df.select_dtypes(include=["object", "category"]).columns)
        for c in df.select_dtypes(include=["int64", "int32", "int16", "int8"]).columns:
            if df[c].nunique() <= 20:
                cat_cols.append(c)
        cat_cols = sorted(set(cat_cols))
        num_cols = [c for c in df.columns if c not in cat_cols]
        return num_cols, cat_cols

    num_cols, cat_cols = infer_columns(X)

    # ---- Preprocessing ----
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    pre = ColumnTransformer(
        transformers=[
            ("num", SimpleImputer(strategy="median"), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
        ],
        remainder="drop"
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc,
        test_size=args.test_size,
        random_state=args.seed,
        stratify=y_enc
    )

    X_train_pre = pre.fit_transform(X_train).astype(np.float64)
    X_test_pre = pre.transform(X_test).astype(np.float64)

    # ---- Fit GAM ----
    print("Fitting GAM model...")
    from pygam import LogisticGAM
    gam = LogisticGAM(n_splines=5, max_iter=200).fit(X_train_pre, y_train)

    # ---- Evaluate ----
    y_pred = gam.predict(X_test_pre)
    from sklearn.metrics import classification_report, confusion_matrix
    print("=== Classification Report (test set) ===")
    print(classification_report(y_test, y_pred, digits=3, target_names=label_enc.classes_))

    cm = confusion_matrix(y_test, y_pred)
    # === ⭐ KAPPA SCORE EKLENDİ ⭐ ===
    kappa = cohen_kappa_score(y_test, y_pred)
    print(f"\n⭐ Cohen Kappa Score: {kappa:.4f}")
    plot_confusion_matrix(cm, label_enc.classes_, args.cm_out)
    print(f"🖼️ Saved confusion matrix to {args.cm_out}")

    # ---- Predict full dataset ----
    X_full_pre = pre.transform(X).astype(np.float64)
    probs = gam.predict_proba(X_full_pre)

    out = df_binary.copy()
    out["prediction"] = label_enc.inverse_transform(gam.predict(X_full_pre))
    if probs.ndim == 1:
        probs = np.column_stack([1 - probs, probs])
    for i, c in enumerate(label_enc.classes_):
        out[f"p_{c}"] = probs[:, i]
    out.to_csv(args.pred_out, index=False)
    print(f"✅ Wrote predictions to {args.pred_out}")

    # ---- Save model ----
    import joblib
    joblib.dump({"model": gam, "preprocess": pre, "label_encoder": label_enc}, args.save_model)
    print(f"💾 Saved GAM model to {args.save_model}")
try:
    main() 
except Exception as e: 
    print("ERROR inside main():", e) 
    raise


Loading CSV from: /Users/defneelagoz/Desktop/practical course/practical-course/predictive_model/gam/student_data.csv
Loaded 4424 rows.
Detected target column: Output
Filtered to binary target: 3630 rows
Fitting GAM model...
=== Classification Report (test set) ===
              precision    recall  f1-score   support

     Dropout      0.922     0.838     0.878       284
    Graduate      0.902     0.955     0.927       442

    accuracy                          0.909       726
   macro avg      0.912     0.896     0.903       726
weighted avg      0.910     0.909     0.908       726


⭐ Cohen Kappa Score: 0.8060
🖼️ Saved confusion matrix to /Users/defneelagoz/Desktop/practical course/practical-course/predictive_model/gam/gam_confusion_matrix.png
✅ Wrote predictions to /Users/defneelagoz/Desktop/practical course/practical-course/predictive_model/gam/gam_predictions.csv
💾 Saved GAM model to /Users/defneelagoz/Desktop/practical course/practical-course/predictive_model/gam/gam_model.jobli